# AWS AI Services APIs

## Install AWS Python SDK client

In [ ]:
!python -m pip install --upgrade boto3
!python -m pip install amazon-textract-response-parser --upgrade
!python -m pip install amazon-textract-prettyprinter


## Import required packages

In [ ]:
# Packages import
import boto3
import time
import json
import uuid
from urllib.request import urlopen

#Textract Libraries for parsing results
from trp import Document
from textractprettyprinter.t_pretty_print import Pretty_Print_Table_Format, Textract_Pretty_Print, get_string

# Multimedia Transcription Exercise

## Amazon Transcribe client

In [ ]:
transcribe = boto3.client('transcribe')

## Specify an audio file to transcribe and generate a UUID as a job name

In [ ]:
media_uri = "s3://m2c-gps304-workshop-us-east-1/lab1-assets/AWS_Podcast_Episode_317_demo.mp3"


## Start the audio file transcription job

In [ ]:
job_name = str(uuid.uuid4())

transcribe.start_transcription_job(
    TranscriptionJobName = job_name,
    Media = {'MediaFileUri': media_uri},
    MediaFormat='mp3',
    LanguageCode='en-US'
)

## Verify the transcription job progress

In [ ]:
while True:
    status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        print(f"Transcription Status: {status['TranscriptionJob']['TranscriptionJobStatus']}")
        break
    print(f"Transcription Status: {status['TranscriptionJob']['TranscriptionJobStatus']}")
    time.sleep(30)

## Get the audio transcription

In [ ]:
transcription_json = json.loads(urlopen(status['TranscriptionJob']['Transcript']['TranscriptFileUri']).read().decode('utf-8'))

In [ ]:
original_transcription = transcription_json['results']['transcripts'][0]['transcript']
print(f"{original_transcription}")

# Improving the original transcription with a custom vocabulary

The description to create a table-based custom vocabulary is described in the [Transcribe documentation](https://docs.aws.amazon.com/transcribe/latest/dg/how-vocabulary.html)

To make things simpler, we already prepared a tab-separated file to create a table-based custom vocabulary:

In [ ]:
vocabulary_name = str(uuid.uuid4())
response = transcribe.create_vocabulary(
    VocabularyName= vocabulary_name,
    LanguageCode='en-US',
    VocabularyFileUri='s3://m2c-gps304-workshop-us-east-1/lab1-assets/custom_vocab.txt'
)

## Verify the custom vocabulary  creation job progress

In [ ]:
while True:
    response = transcribe.get_vocabulary(
        VocabularyName=vocabulary_name
    )    
    status = response['VocabularyState']
    if status in ['READY', 'FAILED']:
        print(response)
        break
    print("Not ready yet...")
    time.sleep(30)

## Transcribe the original  file using the custom vocabulary

In [ ]:
job_name = str(uuid.uuid4())

transcribe.start_transcription_job(
    TranscriptionJobName = job_name,
    Media = {'MediaFileUri': media_uri},
    MediaFormat='mp3',
    LanguageCode='en-US',
    Settings={
        'VocabularyName': vocabulary_name
    }
)

In [ ]:
while True:
    status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        print(f"Transcription Status: {status['TranscriptionJob']['TranscriptionJobStatus']}")
        break
    print(f"Transcription Status: {status['TranscriptionJob']['TranscriptionJobStatus']}")
    time.sleep(15)
    
transcription = json.loads(urlopen(status['TranscriptionJob']['Transcript']['TranscriptFileUri']).read().decode('utf-8'))
new_transcription = transcription['results']['transcripts'][0]['transcript']

## Show the enhanced transcription

In [ ]:
print(f"{new_transcription}")

# Natural Language Processing (NLP) with Amazon Comprehend Exercise

## Create an Amazon Comprehend client

In [ ]:
comprehend = boto3.client('comprehend')

## Entities Detection

In [ ]:
comprehendResponse = comprehend.detect_entities(Text=new_transcription, LanguageCode='en')

for i in range(len(comprehendResponse["Entities"])):
    entity = comprehendResponse["Entities"][i]
    print(f"Entity Type:{entity['Type']} Entity Text:{entity['Text']}")

## Sentiment detection

In [ ]:
comprehendResponse = comprehend.detect_sentiment(Text=new_transcription, LanguageCode='en')
comprehendResponse['Sentiment']

## Key phrases detection

In [ ]:
comprehendResponse = comprehend.detect_key_phrases(Text=new_transcription, LanguageCode='en')

for i in range(len(comprehendResponse["KeyPhrases"])):
    key_phrase = comprehendResponse["KeyPhrases"][i]
    print(f"Key Phrase:{key_phrase['Text']}")

# Intelligent Document Processing (IDP) with Amazon Textract Exercise

## Document to analyze

![Document to Analyze](sampledoc.png)

## Create an Amazon Textract client

In [ ]:
textract = boto3.client('textract')

## Get the document file bytes as a bytearray

In [ ]:
documentName = "sampledoc.png"
with open(documentName, 'rb') as document:
    imageBytes = bytearray(document.read())

## Call Amazon Textract AnalyzeDocument sychronous API

In [ ]:
# Call Amazon Textract
response = textract.analyze_document(Document={'Bytes': imageBytes},FeatureTypes=["FORMS", "TABLES"])

## Use the Textract pretty printer tool to show the form values detected as key-value pairs

In [ ]:
pretty_printed_string = get_string(textract_json=response, output_type=[Textract_Pretty_Print.FORMS], table_format=Pretty_Print_Table_Format.fancy_grid)
print(pretty_printed_string)

## Use the Textract pretty printer tool to show the tables detected

In [ ]:
pretty_printed_string = get_string(textract_json=response, output_type=[Textract_Pretty_Print.TABLES], table_format=Pretty_Print_Table_Format.fancy_grid)
print(pretty_printed_string)

## Use the Textract pretty printer tool to show the text detected

In [ ]:
pretty_printed_string = get_string(textract_json=response, output_type=[Textract_Pretty_Print.LINES], table_format=Pretty_Print_Table_Format.fancy_grid)
print(pretty_printed_string)

## Amazon Comprehend entities detection on document text detected by Amazon Textract

In [ ]:
comprehendResponse = comprehend.detect_entities(Text=pretty_printed_string, LanguageCode='en')

for i in range(len(comprehendResponse["Entities"])):
    entity = comprehendResponse["Entities"][i]
    print(f"Entity Type:{entity['Type']} Entity Text:{entity['Text']}")